# 🥉 Bronze Layer — Raw Ingestion

**Medallion Architecture: Bronze → Silver → Gold**

This notebook ingests the 6 raw CSV files provided for the Annie's Magic Numbers
challenge into Delta tables with zero business transformation.

| Source File | Bronze Table |
|---|---|
| BegInvFINAL12312016.csv | bronze.beg_inventory |
| EndInvFINAL12312016.csv | bronze.end_inventory |
| PurchasesFINAL12312016.csv | bronze.purchases |
| SalesFINAL12312016.csv | bronze.sales |
| InvoicePurchases12312016.csv | bronze.invoice_purchases |
| 2017PurchasePricesDec.csv | bronze.purchase_prices |

---
**Author:** Data Engineering Team  
**Last Updated:** 2026-02-24  
**Run Order:** 1 of 3

## 0. Configuration

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType
from datetime import datetime

# ---------------------------------------------------------------------------
# CONFIG — Update DBFS_ROOT to wherever you uploaded the files in your workspace
# Recommended: /Volumes/<catalog>/<schema>/raw_files/  (Unity Catalog Volume)
#              or /FileStore/annie_magic_numbers/ (legacy DBFS)
# ---------------------------------------------------------------------------
DBFS_ROOT    = "/FileStore/annie_magic_numbers"   # ← UPDATE THIS PATH
BRONZE_DB    = "bronze"
CATALOG      = "main"                             # ← UPDATE if using Unity Catalog

# Fully-qualified database name
BRONZE_SCHEMA = f"{CATALOG}.{BRONZE_DB}"

# Ingestion timestamp (consistent across all tables in a single run)
RUN_TS = datetime.utcnow().isoformat()

print(f"Source path  : {DBFS_ROOT}")
print(f"Target schema: {BRONZE_SCHEMA}")
print(f"Run timestamp: {RUN_TS}")

## 1. Create Bronze Database / Schema

In [ ]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {BRONZE_SCHEMA}")
print(f"✅  Schema '{BRONZE_SCHEMA}' is ready.")

## 2. Helper — Generic CSV → Delta Ingestion

In [ ]:
def ingest_csv_to_bronze(
    source_path: str,
    target_table: str,
    *,
    header: bool = True,
    infer_schema: bool = True,
    multiline: bool = False
) -> None:
    """
    Read a CSV from DBFS and write it as a Delta table in the Bronze schema.

    Args:
        source_path  : Full DBFS path to the CSV file.
        target_table : Fully-qualified target Delta table (schema.table).
        header       : Whether the CSV has a header row.
        infer_schema : If True, Spark infers column types; otherwise all strings.
        multiline    : Set True for CSVs that may contain embedded newlines.
    """
    print(f"\n📥  Ingesting  {source_path}")
    print(f"       → {target_table}")

    df = (
        spark.read
             .option("header", str(header).lower())
             .option("inferSchema", str(infer_schema).lower())
             .option("multiLine", str(multiline).lower())
             .option("escape", '"')
             .option("quote", '"')
             .option("encoding", "UTF-8")
             .csv(source_path)
    )

    # Add metadata columns required by the Medallion pattern
    df = df.withColumn("_ingestion_timestamp", F.lit(RUN_TS)) \
           .withColumn("_source_file", F.lit(source_path))

    row_count = df.count()
    print(f"       rows read : {row_count:,}")

    # Write as Delta — overwrite ensures idempotency on re-runs
    (
        df.write
          .format("delta")
          .mode("overwrite")
          .option("overwriteSchema", "true")
          .saveAsTable(target_table)
    )

    print(f"   ✅  Written to {target_table}  ({row_count:,} rows)")

## 3. Ingest Each Source File

In [ ]:
# 3.1 — Beginning Inventory (Jan 1, 2016)
ingest_csv_to_bronze(
    source_path  = f"{DBFS_ROOT}/BegInvFINAL12312016.csv",
    target_table = f"{BRONZE_SCHEMA}.beg_inventory"
)

In [ ]:
# 3.2 — Ending Inventory (Dec 31, 2016)
ingest_csv_to_bronze(
    source_path  = f"{DBFS_ROOT}/EndInvFINAL12312016.csv",
    target_table = f"{BRONZE_SCHEMA}.end_inventory"
)

In [ ]:
# 3.3 — All Purchase Transactions 2016 (large file ~400 MB)
ingest_csv_to_bronze(
    source_path  = f"{DBFS_ROOT}/PurchasesFINAL12312016.csv",
    target_table = f"{BRONZE_SCHEMA}.purchases"
)

In [ ]:
# 3.4 — All Sales Transactions 2016 (largest file ~1.7 GB)
ingest_csv_to_bronze(
    source_path  = f"{DBFS_ROOT}/SalesFINAL12312016.csv",
    target_table = f"{BRONZE_SCHEMA}.sales"
)

In [ ]:
# 3.5 — Invoice-level Purchase Summaries
ingest_csv_to_bronze(
    source_path  = f"{DBFS_ROOT}/InvoicePurchases12312016.csv",
    target_table = f"{BRONZE_SCHEMA}.invoice_purchases"
)

In [ ]:
# 3.6 — 2017 Reference Purchase Prices (cost basis)
ingest_csv_to_bronze(
    source_path  = f"{DBFS_ROOT}/2017PurchasePricesDec.csv",
    target_table = f"{BRONZE_SCHEMA}.purchase_prices"
)

## 4. Validation Checks

In [ ]:
tables = [
    "beg_inventory",
    "end_inventory",
    "purchases",
    "sales",
    "invoice_purchases",
    "purchase_prices",
]

print("\n📊  Bronze Layer Validation Report")
print("=" * 55)

for tbl in tables:
    full_name = f"{BRONZE_SCHEMA}.{tbl}"
    count = spark.table(full_name).count()
    col_count = len(spark.table(full_name).columns)
    print(f"  {tbl:<25} | {count:>12,} rows | {col_count} cols")

print("=" * 55)
print("✅  All Bronze tables successfully ingested.\n")

## 5. Schema Preview

Quick visual confirmation of the Bronze schemas.

In [ ]:
for tbl in tables:
    print(f"\n{'='*50}")
    print(f"  {BRONZE_SCHEMA}.{tbl}")
    print(f"{'='*50}")
    spark.table(f"{BRONZE_SCHEMA}.{tbl}").printSchema()

## ✅ Bronze Ingestion Complete

**Next step:** Run `02_silver_cleaning.ipynb` to clean, cast types, and enrich the data.